In [146]:
import pandas as pd
import sys
import os

# Add the parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join('..')))

ENV_NAME = "MOLavaGridDR-v0" # CHANGE THIS TO THE NAME OF THE ENVIRONMENT
SEEDS = [5,26,47,76,92] # CHANGE THIS TO THE SEEDS YOU USE

from helpers.utils import ENVIRONMENTS_MAP, DOMAINS_REFERENCE_POINTS, get_algorithms
ALGORITHMS = get_algorithms(ENV_NAME)
REFERENCE_POINT = DOMAINS_REFERENCE_POINTS[ENV_NAME]
REWARD_DIM = len(REFERENCE_POINT)

### Normalize Front and Calculate Normalized Hypervolume and EUM for Generalist and Specialist

Import helpers

In [147]:
import numpy as np

sys.path.append(os.path.abspath(os.path.join('../../..')))
sys.path.append(os.path.abspath(os.path.join('../..')))

def get_normalized_vec_returns(all_vec_returns, minmax_range):
    minmax_array = np.array([minmax_range[str(i)] for i in range(all_vec_returns.shape[-1])])
    min_vals = minmax_array[:, 0].reshape(1, 1, -1) # reshape to (1, 1, n_objectives) for broadcasting
    max_vals = minmax_array[:, 1].reshape(1, 1, -1)

    clipped_vec_returns = np.clip(all_vec_returns, min_vals, max_vals) # broadcasted clipping
    
    # Normalize
    normalized_vec_returns = (clipped_vec_returns - min_vals) / (max_vals - min_vals)
    
    return normalized_vec_returns

In [148]:
from mo_utils.performance_indicators import hypervolume, expected_utility
from mo_utils.weights import equally_spaced_weights

NUM_WEIGHTS = 100 # CHANGE THIS TO THE NUMBER OF WEIGHTS YOU WANT TO USE, NORMALLY ITS 100, FOR MARIO ITS 32
EVAL_WEIGHTS = equally_spaced_weights(REWARD_DIM, NUM_WEIGHTS) 

EVAL_WEIGHTS[0:5]

[array([0., 1.]),
 array([0.01002355, 0.98997645]),
 array([0.02012817, 0.97987183]),
 array([0.03023213, 0.96976787]),
 array([0.04032552, 0.95967448])]

### Combine the fronts of all the Specialists for each environment

Skip this step if already done before

In [149]:
import glob
import warnings
from mo_utils.pareto import filter_pareto_dominated

curr_envs = ENVIRONMENTS_MAP[ENV_NAME]
SPECIALIST_FRONT = "eval/front" # don't change this, this is the discounted fronts but poorly named!!
path_to_find_fronts = f"../data/single_env/{SPECIALIST_FRONT}/{ENV_NAME}"

for env in curr_envs:
    sub_env_folder = os.path.join(path_to_find_fronts, env)
    csv_files = glob.glob(os.path.join(sub_env_folder, "*.csv"))

    if not csv_files:
        warnings.warn(f"No fronts found for {env}")
        continue
    
    unfiltered_combined_front_df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
    print(f"Combined front for {env} has {len(unfiltered_combined_front_df)} rows")

    # for column in unfiltered_combined_front_df.columns:
    #     min_value = unfiltered_combined_front_df[column].min()
    #     max_value = unfiltered_combined_front_df[column].max()
    #     print(f"{column}, Min: {min_value}, Max: {max_value}")

    combined_front_array = unfiltered_combined_front_df.to_numpy()
    filtered_combined_front_array = filter_pareto_dominated(combined_front_array)

    combined_front_df = pd.DataFrame(filtered_combined_front_array, columns=unfiltered_combined_front_df.columns)
    print(f"Filtered front for {env} has {len(combined_front_df)} rows")
    save_dir = f"../data/single_env/combined_fronts/{ENV_NAME}/"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    combined_front_df.to_csv(f"{save_dir}/{env}.csv", index=False)

Combined front for MOLavaGridCheckerBoard-v0 has 1 rows
Filtered front for MOLavaGridCheckerBoard-v0 has 1 rows
Combined front for MOLavaGridSmiley-v0 has 3 rows
Filtered front for MOLavaGridSmiley-v0 has 3 rows
Combined front for MOLavaGridSnake-v0 has 2 rows
Filtered front for MOLavaGridSnake-v0 has 2 rows
Combined front for MOLavaGridIslands-v0 has 2 rows
Filtered front for MOLavaGridIslands-v0 has 2 rows
Combined front for MOLavaGridLabyrinth-v0 has 3 rows
Filtered front for MOLavaGridLabyrinth-v0 has 3 rows
Combined front for MOLavaGridMaze-v0 has 5 rows
Filtered front for MOLavaGridMaze-v0 has 5 rows
Combined front for MOLavaGridCorridor-v0 has 6 rows
Filtered front for MOLavaGridCorridor-v0 has 6 rows
Combined front for MOLavaGridRoom-v0 has 7 rows
Filtered front for MOLavaGridRoom-v0 has 7 rows


### Find boundary values across ALL fronts

In [150]:
import json
from pathlib import Path

all_minmax_values = {}
for env in curr_envs:
    # get paths to all the fronts
    fronts = [f'../data/single_env/combined_fronts/{ENV_NAME}/{env}.csv'] # specialist front
    for algo in ALGORITHMS:
        for seed in SEEDS:
            fronts.append(f'../data/eval/discounted_front/{ENV_NAME}/{algo}/seed_{seed}/{env}.csv')

    for f in fronts:
        assert Path(f).exists(), f"Error: File {f} does not exist"
    
    combined_front_df = pd.concat([pd.read_csv(f) for f in fronts], ignore_index=True)

    print(f"Combined front for {env} has {len(combined_front_df)} rows")
    
    
    print(f"Minimum and maximum values for {env}")
    env_minmax_values = {}
    for i, column in enumerate(combined_front_df.columns):
        min_value = combined_front_df[column].min()
        max_value = combined_front_df[column].max()
        print(f"{column}, Min: {min_value}, Max: {max_value}")

        env_minmax_values[i] = [min_value, max_value]

    all_minmax_values[env] = env_minmax_values

save_dir = f"../data/minmax_values/"
os.makedirs(save_dir, exist_ok=True)

# Define the path for saving JSON
save_path = os.path.join(save_dir, f"{ENV_NAME}.json")

# Save min-max values as JSON
with open(save_path, "w") as json_file:
    json.dump(all_minmax_values, json_file, indent=4)

print(f"Min-max values saved to {save_path}")

Combined front for MOLavaGridCheckerBoard-v0 has 48 rows
Minimum and maximum values for MOLavaGridCheckerBoard-v0
objective_1, Min: -1323.0126867369104, Max: 107.33586502075195
objective_2, Min: -433.71237854709295, Max: 218.76215887069705
Combined front for MOLavaGridSmiley-v0 has 51 rows
Minimum and maximum values for MOLavaGridSmiley-v0
objective_1, Min: -1032.8942598727158, Max: 270.69044494628906
objective_2, Min: -433.71237854709295, Max: 225.50174689292908
Combined front for MOLavaGridSnake-v0 has 44 rows
Minimum and maximum values for MOLavaGridSnake-v0
objective_1, Min: -860.8560231233766, Max: 234.21388816833496
objective_2, Min: -345.93999076612414, Max: 220.5411500930786
Combined front for MOLavaGridIslands-v0 has 61 rows
Minimum and maximum values for MOLavaGridIslands-v0
objective_1, Min: -1691.1169121283383, Max: 124.23237419128418
objective_2, Min: -433.71237854709295, Max: 204.70165252685547
Combined front for MOLavaGridLabyrinth-v0 has 53 rows
Minimum and maximum valu

### Normalize the fronts and calculate normalized hypervolume and EUM for SPECIALIST

In [151]:
import json

save_dir = f"../data/minmax_values/"
save_path = os.path.join(save_dir, f"{ENV_NAME}.json")

with open(save_path, "r") as json_file:
    normalization_data = json.load(json_file)

print("Loaded min-max values:", normalization_data)

Loaded min-max values: {'MOLavaGridCheckerBoard-v0': {'0': [-1323.0126867369104, 107.33586502075195], '1': [-433.71237854709295, 218.76215887069705]}, 'MOLavaGridSmiley-v0': {'0': [-1032.8942598727158, 270.69044494628906], '1': [-433.71237854709295, 225.50174689292908]}, 'MOLavaGridSnake-v0': {'0': [-860.8560231233766, 234.21388816833496], '1': [-345.93999076612414, 220.5411500930786]}, 'MOLavaGridIslands-v0': {'0': [-1691.1169121283383, 124.23237419128418], '1': [-433.71237854709295, 204.70165252685547]}, 'MOLavaGridLabyrinth-v0': {'0': [-1414.3869692886285, 250.0454330444336], '1': [-303.3670912278126, 226.9515025615692]}, 'MOLavaGridMaze-v0': {'0': [-1526.3784222067, 237.33114337921145], '1': [-433.71237854709295, 203.5929992198944]}, 'MOLavaGridCorridor-v0': {'0': [-793.4801398683029, 265.6644287109375], '1': [-433.71237854709295, 240.20966625213623]}, 'MOLavaGridRoom-v0': {'0': [-1221.0622601697146, 263.860782623291], '1': [-433.71237854709295, 215.7504994869232]}}


In [152]:
from helpers.utils import ENVIRONMENTS_MAP

FRONT = "eval/discounted_front" # don't change this, front extracted for specialists are only the discounted ones!!
file_path = f"../data/{FRONT}/{ENV_NAME}"
scores_save_path = f"../data/scores/{ENV_NAME}"

os.makedirs(f"{scores_save_path}", exist_ok=True)

In [153]:
specialist_hypervolumes = []
specialist_eums = []
normalized_specialist_hypervolumes = []

for env in ENVIRONMENTS_MAP[ENV_NAME]:
    min_max_ranges = normalization_data[env]
    best_env_front_path = f"../data/single_env/combined_fronts/{ENV_NAME}/{env}.csv"
    assert os.path.exists(best_env_front_path), f"File {best_env_front_path} does not exist"
    
    best_env_front = pd.read_csv(best_env_front_path)
    data_array = best_env_front.to_numpy()
    specialist_hypervolumes.append(hypervolume(np.array(REFERENCE_POINT), data_array))
    normalized_front = get_normalized_vec_returns(data_array, min_max_ranges)
    normalized_specialist_hypervolumes.append(hypervolume(np.zeros(REWARD_DIM), normalized_front[0]))
    specialist_eums.append(expected_utility(data_array, weights_set=EVAL_WEIGHTS))

specialist_data = {f"hypervolume/{env}": [specialist_hypervolumes[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])}
specialist_data.update({f"normalized_hypervolume/{env}": [normalized_specialist_hypervolumes[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])})
specialist_data.update({f"eum/{env}": [specialist_eums[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])})
all_specialist_data = pd.DataFrame(specialist_data).T
all_specialist_data.columns = ["score"]
all_specialist_data.to_csv(f"{scores_save_path}/specialist.csv", index=True, index_label="metric")

### Normalize the fronts and calculate normalized hypervolume and EUM for GENERALIST

In [154]:
# Load the data
for algo in ALGORITHMS:
    for seed in SEEDS:
        hypervolumes = []
        eums = []
        normalized_hypervolumes = []
        for env in ENVIRONMENTS_MAP[ENV_NAME]:
            min_max_ranges = normalization_data[env]
            file = f"{file_path}/{algo}/seed_{seed}/{env}.csv"
            assert os.path.exists(file), f"File {file} does not exist"
            data = pd.read_csv(file)
            # Convert dataframe to numpy array of vectors
            data_array = data.to_numpy()
            normalized_front = get_normalized_vec_returns(data_array, min_max_ranges)
            hypervolumes.append(hypervolume(np.array(REFERENCE_POINT), data_array))
            normalized_hypervolumes.append(hypervolume(np.zeros(REWARD_DIM), normalized_front[0]))
            eums.append(expected_utility(data_array, weights_set=EVAL_WEIGHTS))

        data = {f"hypervolume/{env}": [hypervolumes[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])}
        data.update({f"normalized_hypervolume/{env}": [normalized_hypervolumes[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])})
        data.update({f"eum/{env}": [eums[i]] for i, env in enumerate(ENVIRONMENTS_MAP[ENV_NAME])})
        df = pd.DataFrame(data).T
        df.columns = ["score"]
        os.makedirs(f"{scores_save_path}/{algo}/", exist_ok=True)
        df.to_csv(f"{scores_save_path}/{algo}/seed_{seed}.csv", index=True, index_label="metric")
            

### Calculate NHGR and EUGR

In [155]:
# get the scores of the specialists
specialist_scores = pd.read_csv(f"{scores_save_path}/specialist.csv", index_col=[0])
specialist_scores

,score
metric,
hypervolume/MOLavaGridCheckerBoard-v0,795911.116937
hypervolume/MOLavaGridSmiley-v0,921329.875304
hypervolume/MOLavaGridSnake-v0,888921.260241
hypervolume/MOLavaGridIslands-v0,792119.076773
hypervolume/MOLavaGridLabyrinth-v0,907746.691976
hypervolume/MOLavaGridMaze-v0,868118.169978
hypervolume/MOLavaGridCorridor-v0,932332.899298
hypervolume/MOLavaGridRoom-v0,902388.601158
normalized_hypervolume/MOLavaGridCheckerBoard-v0,1.000000


In [156]:
for algo in ALGORITHMS:
    for seed in SEEDS:
        # get the normalized hypervolumes we extracted earlier
        file = f"{scores_save_path}/{algo}/seed_{seed}.csv"
        seed_scores_data = pd.read_csv(file, index_col=[0])

        # create new rows for NHGR and EUGR
        nhgr_values = {}
        eugr_values = {}

        for env in ENVIRONMENTS_MAP[ENV_NAME]:
            # Filter columns that start with "normalized_hypervolume"
            specialist_normalized_hv = specialist_scores.loc[f'normalized_hypervolume/{env}'].values[0]
            generalist_normalized_hv = seed_scores_data.loc[f'normalized_hypervolume/{env}'].values[0]

            specialist_eum = specialist_scores.loc[f'eum/{env}'].values[0]
            generalist_eum = seed_scores_data.loc[f'eum/{env}'].values[0]
            
            # compute NHGR and EUGR
            env_nhgr = min(generalist_normalized_hv / specialist_normalized_hv, 1.0)
            env_eugr = min(max(generalist_eum / specialist_eum, 0), 1)  # Clamp between [0,1]

            nhgr_values[f'NHGR/{env}'] = env_nhgr
            eugr_values[f'EUGR/{env}'] = env_eugr
        
        # convert new NHGR and EUGR rows into DataFrame
        nhgr_df = pd.DataFrame.from_dict(nhgr_values, orient='index', columns=['score'])
        eugr_df = pd.DataFrame.from_dict(eugr_values, orient='index', columns=['score'])

        # Append the new rows to the existing data
        updated_seed_scores_data = pd.concat([seed_scores_data, nhgr_df, eugr_df])

        # Save the updated DataFrame, ensuring the index is retained
        updated_seed_scores_data.to_csv(f"{scores_save_path}/{algo}/seed_{seed}.csv", index=True, index_label="metric")

### Plot table of all metrics

In [157]:
algo_results = {algo: {"hypervolume": [], "eum": [], "NHGR": [], "EUGR": []} for algo in ALGORITHMS}

# Read data and aggregate per algorithm
for algo in ALGORITHMS:
    for seed in SEEDS:
        file = f"{scores_save_path}/{algo}/seed_{seed}.csv"
        seed_scores_data = pd.read_csv(file, index_col=[0])

        # Extract values for each metric
        for metric in ["hypervolume", "eum", "NHGR", "EUGR"]:
            metric_values = [
                seed_scores_data.loc[f"{metric}/{env}"].values[0]
                for env in ENVIRONMENTS_MAP[ENV_NAME] if f"{metric}/{env}" in seed_scores_data.index
            ]
            if metric_values:  # Ensure we don't append empty lists
                algo_results[algo][metric].append(metric_values)

# Compute mean and standard deviation
algo_means = {algo: {m: np.mean(v, axis=1) for m, v in results.items()} for algo, results in algo_results.items()}
algo_stds = {algo: {m: np.std(v, axis=1) for m, v in results.items()} for algo, results in algo_results.items()}


algo_stats = {}
for algo in ALGORITHMS:
    algo_stats[algo] = {
        metric: f"{np.mean(values):.4f} ± {np.std(values):.4f}" if values else "N/A"
        for metric, values in algo_results[algo].items()
    }

results_df = pd.DataFrame.from_dict(algo_stats, orient="index").T

results_df

,MORL-D(MOSACDiscrete)-SB+PSA,MORL-D(MOSACDiscrete)-SB,GPI-LS,Envelope,PCN,SAC Discrete Action
hypervolume,275934.8234 ± 184468.2656,354158.4777 ± 187689.1083,111841.0373 ± 57583.6199,109675.7156 ± 56727.9094,111907.6674 ± 67093.3477,203898.6175 ± 193707.1843
eum,-133.7638 ± 105.0942,-84.7980 ± 95.3558,-197.4365 ± 51.1552,-194.8267 ± 47.9604,-206.4874 ± 60.7722,-350.3699 ± 230.7719
NHGR,0.2716 ± 0.2281,0.3711 ± 0.2124,0.0236 ± 0.0499,0.0202 ± 0.0466,0.0268 ± 0.0575,0.2525 ± 0.1922
EUGR,0.0413 ± 0.1299,0.0602 ± 0.1455,0.0000 ± 0.0000,0.0000 ± 0.0000,0.0000 ± 0.0000,0.0391 ± 0.1403
